In [15]:
import pandas as pd
import numpy as np

import re


In [4]:
df=pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df.isnull().sum()

,0
Category,0
Message,0


In [9]:
df.duplicated().sum()

415

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df['Category'].unique()

array(['ham', 'spam'], dtype=object)

In [13]:
df['Category']=df['Category'].map({'ham':0,'spam':1})

In [14]:
df['Category'].value_counts()

,count
Category,
0,4516
1,641


In [16]:
df["Message"] = df["Message"].str.lower()


In [17]:
df["Message"] = df["Message"].apply(lambda x: re.sub(r"[^a-z0-9\s]", "", x))


In [18]:
df["Message"] = df["Message"].str.strip()
